In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

# os.environ["OPENAI_MODEL_NAME"]="gpt-4-turbo-preview"
# os.environ["OPENAI_MODEL_NAME"]="gpt-3.5-turbo"
os.environ["OPENAI_MODEL_NAME"]="gpt-4-turbo"

search_tool = SerperDevTool()

# My Agents

In [3]:
# General Fighter Stat Analysis Agent
agent1 = Agent(
    role="Fighter Stats Analyst",
    goal="""Analyze fighter's performance statistics and evaluate strengths and
    weaknesses based on fighter data""",
    backstory=
        """As a seasoned analyst with a deep understanding of MMA/UFC, you specialize
        in breaking down fights and understanding fighter tendencies and histories.""",
    max_iter=10,
    allow_delegation=True,
    tools=[search_tool],
)

# Recent Performance and Momentum Analysis Agent
agent2 = Agent(
    role="Historical Fight Analyst",
    goal="""Analyze previous fights to learn performance against similar opponents 
    as well to assess current momentum of the fighter""",
    backstory=
        """As a seasoned analyst with a deep understanding of MMA/UFC, you specialize
        in breaking down fights.
        Your expertise lies in dissecting historical fight outcomes and statistics,
        providing insights into how a fighter's recent experiences might influence their
        next performance.""",
    max_iter=10,
    allow_delegation=True,
    tools=[search_tool],
)

# Public Sentiment Analysis Agent
agent3 = Agent(
    role="Public Sentiment Analyst",
    goal="""Analyze public sentiment from various online sources to gauge who the public 
    believes will win upcoming UFC fights. This helps in understanding betting trends and 
    public biases, which can inform more strategic handicapping decisions.""",
    backstory=
        """As a data-driven sentiment analyst with a focus on combat sports, you specialize
        in extracting and interpreting the mood and opinions from social media, forums, and 
        discussion boards. Your analysis provides a comprehensive view of public opinion, 
        which is crucial for predicting fight outcomes and understanding public perception 
        dynamics in the UFC.""",
    max_iter=10,
    allow_delegation=True,
    tools=[search_tool],
)

# Odds Maker Agent
agent4 = Agent(
    role="Odds Maker",
    goal="Generate the American betting lines for upcoming UFC fights.",
    backstory=
        """As an expert analyst and oddsmaker with a deep understanding of MMA/UFC,
        you specialize in breaking down and handicapping fights. You use professional
        handicapping knowledge and metrics to assist in your betting line generations.
        You consider various factors such as fighter statistics, historical performance,
        recent momentum, public sentiment, and expert predictions to create accurate
        betting lines that reflect the true probabilities of fight outcomes.""",
    allow_delegation=True,
    tools=[search_tool],
)

# Predictor Agent
agent_final = Agent(
    role="Prediction Agent",
    goal="Predict outcomes of upcoming fights based on statistical analysis",
    backstory=
        """As a expert analyst and oddsmaker with a deep understanding of MMA/UFC,
        you specialize in breaking down and handicapping fights.
        Equipped with advanced predictive modeling and professional handicapping knowledge,
        you use and evaluate data to predict hypothetical fight outcomes with high accuracy.""",
    allow_delegation=True,
)

agents = [agent1, agent2, agent3, agent4, agent_final]

In [4]:
def ufc_agent_loop(fighter1, fighter2, agents):
    agent1, agent2, agent3, agent4, agent_final = agents
    tasks = [
        Task(
            description=f"Research detailed statistics and historical performance of {fighter1}",
            expected_output=f"Comprehensive statistical report on {fighter1}.",
            agent=agent1,
#             max_iter=15,
            output_file=f'./logs/{fighter1}_stats.txt'
        ),
        Task(
            description=f"Research detailed statistics and historical performance of {fighter2}",
            expected_output=f"Comprehensive statistical report on {fighter2}.",
            agent=agent1,
#             max_iter=15, 
            output_file=f'./logs/{fighter2}_stats.txt'
        ),
        Task(
            description=f"Analyze historical fight data and momentum of {fighter1}",
            expected_output=f"Detailed analysis of fight history and current momentum of {fighter1}.",
            agent=agent2,
#             max_iter=15,
            output_file=f'./logs/{fighter1}_momentum.txt'
        ),
        Task(
            description=f"Analyze historical fight data and momentum of {fighter2}",
            expected_output=f"Detailed analysis of fight history and current momentum of {fighter2}.",
            agent=agent2,
#             max_iter=15,
            output_file=f'./logs/{fighter2}_momentum.txt'
        ),
        Task(
            description=f"Research {fighter1} versus {fighter2}",
            expected_output=f"Comprehensive analysis on the general sentiment and public biases.",
            agent=agent3,
            output_file=f'./logs/{fighter1}_vs_{fighter2}_sentiment.txt'
        ),
        Task(
            description=f"Generate American betting lines for {fighter1} vs {fighter2}.",
            expected_output="American betting lines for the fight, including moneyline odds for both fighters.",
            agent=agent4,
            output_file=f'./logs/{fighter1}_vs_{fighter2}_odds.txt'
        ),
        Task(
            description=f"""Using the insights provided, predict the outcome of a hypothetical
            fight between {fighter1} and {fighter2}.""",
            expected_output=
            """Your prediction should be informative and insightful based on findings from your 
            research. You must include the round and method of victory in your prediction.""",
            agent=agent_final,
            output_file=f'./logs/{fighter1}_vs_{fighter2}_prediction.txt'
        )
    ]

    # Assemble crew with agents and tasks
    crew = Crew(
        agents=[agent1, agent2, agent3, agent4, agent_final],
        tasks=tasks,
        verbose=False
    )

    result = crew.kickoff() 

    with open(f"./fight-predictions/{fighter1}_v_{fighter2}.txt", "a") as file:
        file.write(str(result) + "\n")
        file.write("----------------------------------------\n")


In [5]:
# Define the list of fights
fights = [
    ("Jared Cannonier", "Nassourdine Imavov"),
    ("Dominick Reyes", "Dustin Jacoby"),
    ("Raul Rosas Jr.", "Ricky Turcios"),
    ("Brunno Ferreira", "Dustin Stoltzfus"),
    ("Julian Marquez", "Zachary Reese"),
    ("Miguel Baeza", "Punahele Soriano"),
    ("Thiago Moises", "Ludovit Klein"),
    ("Charlie Radtke", "Carlos Prates"),
    ("Brad Katona", "Jesse Butler"),
    ("Andrea Lee", "Montana De La Rosa"),
    ("John Castaneda", "Daniel Marcos"),
    ("Eduarda Moura", "Denise Gomes"),
    ("Cody Stamann", "Taylor Lapilus"),
    ("Rayanne dos Santos", "Puja Tomar")
]

# Iterate through each fight and call the function
for fighter1, fighter2 in fights:
    ufc_agent_loop(fighter1, fighter2, agents)

 


Search results: Title: Jared Cannonier Record: 17-6-0 - Stats | UFC
Link: http://ufcstats.com/fighter-details/13a0275fa13c4d26
Snippet: Jared Cannonier Record: 17-6-0 ... Career statistics: SLpM: 4.68; Str. Acc.: 52%; SApM: 4.13; Str. Def: 59%.
---
Title: Jared Cannonier (Middleweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/3154860/jared-cannonier
Snippet: View the profile of the MMA fighter Jared Cannonier ... Stats. W-L-D. 17-6-0. (T)KO. 10-2. SUB. 2-0. Overview ... Jared Cannonier, Striker, 5' 11", 185 lbs.
---
Title: Jared Cannonier ("Killa Gorilla") | MMA Fighter Page - Tapology
Link: https://www.tapology.com/fightcenter/fighters/63163-jared-cannonier
Snippet: Jared Cannonier MMA Fight Record. Pro MMA Statistics. Record: 17-6-0. KO/TKO. 9 wins, 2 losses. W. L. 53% of wins. 33% of losses. Submission. 3 wins, 0 losses.
---
Title: Jared Cannonier - The Killa Gorilla - Sherdog
Link: https://www.sherdog.com/fighter/Jared-Cannonier-78628
Snippet: Jared Cannon

 


Search results: Title: Nassourdine Imavov Record: 13-4-0 (1 NC) - Stats | UFC
Link: http://www.ufcstats.com/fighter-details/881bf86d4cba8578
Snippet: Nassourdine Imavov Record: 13-4-0 (1 NC) ; Jared Cannonier · Matchup Preview · UFC Fight Night: Cannonier vs. Imavov. Jun. 08, 2024 ; Roman Dolidze · 112. 34. 0. 0.
---
Title: Nassourdine Imavov - UFC.com
Link: https://www.ufc.com/athlete/nassourdine-imavov?language_content_entity=en
Snippet: Nassourdine Imavov is a French professional mixed martial artist in the UFC middleweight division. Get the latest UFC breaking news, fight ...
---
Title: Nassourdine Imavov - Fighter Profile - Fight Matrix
Link: https://www.fightmatrix.com/fighter-profile/Nassourdine+Imavov/157387/
Snippet: Opponent, Event, Outcome. W, Roman Dolidze #13 Middleweight, UFC Fight Night 235 - Dolidze vs. Imavov Saturday, February 3rd 2024, Decision (Majority)
---
Title: UFC free fight: Nassourdine Imavov defeats Joaquin Buckley in ...
Link: https://sports.yahoo.com/u

 

Based on the gathered statistics:

Jared Cannonier has a professional MMA record of 17 wins, 6 losses, and 0 draws, with 10 wins by knockout, 2 wins by submission, and 5 wins by decision. He currently holds a 2-win streak and is known as the "Killa Gorilla."

Nassourdine Imavov has a professional MMA record of 13 wins, 4 losses, and 0 draws, with 5 wins by knockout, 4 wins by submission, and 4 wins by decision. He is also referred to as the "Russian Sniper."

These statistics provide a solid foundation for analyzing the strengths and weaknesses of both fighters in the upcoming fight.

 


Search results: Title: Jared Cannonier Record: 17-6-0 - Stats | UFC
Link: http://ufcstats.com/fighter-details/13a0275fa13c4d26
Snippet: Jared Cannonier Record: 17-6-0 ... Career statistics: SLpM: 4.68; Str. Acc.: 52%; SApM: 4.13; Str. Def: 59%.
---
Title: Jared Cannonier (Middleweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/3154860/jared-cannonier
Snippet: W-L-D. 17-6-0 ; (T

 


Search results: Title: Nassourdine Imavov (Middleweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/4690539/nassourdine-imavov
Snippet: Stats ; W-L-D. 13-4-0 ; (T)KO. 5-0 ; SUB. 4-1.
---
Title: Nassourdine Imavov MMA Career Stats - ESPN
Link: https://www.espn.com/mma/fighter/stats/_/id/4690539/nassourdine-imavov
Snippet: Complete career stats for the fighter Nassourdine Imavov on ESPN. Includes striking, clinch and ground stats per MMA fight.
---
Title: Nassourdine Imavov MMA Stats, Pictures, News, Videos, Biography
Link: https://www.sherdog.com/fighter/Nassourdine-Imavov-217405
Snippet: Nassourdine Imavov breaking news and and highlights for UFC on ESPN 57 fight vs. Nassourdine Imavov, with official Sherdog mixed martial arts stats, photos, ...
---
Title: Nassourdine Imavov ("Russian Sniper") | MMA Fighter Page
Link: https://www.tapology.com/fightcenter/fighters/125404-nassourdine-imavov
Snippet: Nassourdine Imavov MMA Fight Record ; KO/TKO. 5 wins, 0 losses · 

2024-06-06 16:48:59,354 - 8294452224 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


...

 

Rayanne dos Santos is a Brazilian mixed martial artist competing in the strawweight division with a professional MMA record of 14 wins and 7 losses. She has fought in various organizations including the UFC.

Significant Wins:
- Information not explicitly provided in the search results.

Significant Losses:
- Talita Alencar
- Denise Gomes

Fighting Style:
- Information not explicitly provided in the search results.

Strengths:
- Information not explicitly provided in the search results.

Weaknesses:
- Information not explicitly provided in the search results.

For a more detailed analysis of Rayanne dos Santos' fighting style, strengths, weaknesses, and significant wins, further research and analysis would be required.

 


Search results: Title: Puja Tomar (Women's Strawweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/4292760/puja-tomar
Snippet: View the profile of the MMA fighter Puja Tomar from India on ESPN. Get the latest news, live stats and MMA fight hig

 


Search results: Title: Rayanne Amanda vs Puja Tomar Pick and Predictions #ufcfightnight
Link: https://www.youtube.com/watch?v=j6qNkY0CxF4
Snippet: Rayanne Amanda vs Puja Tomar Pick and Predictions #ufcfightnight. 70 views · 10 hours ago ...
---
Title: Puja Tomar vs Rayanne dos Santos Odds: Spreads, Lines & Totals
Link: https://www.bookmakersreview.com/ufc/matchups/puja-tomar-vs-rayanne-dos-santos/odds/
Snippet: Check out the current UFC odds for the Puja Tomar vs Rayanne dos Santos game on Sat, Jun 8, 2024 including spreads, moneylines and totals offered by the top ...
---
Title: Preview: UFC on ESPN 57 Prelims - Dos Santos vs. Tomar - Sherdog
Link: https://www.sherdog.com/news/articles/8/Preview-UFC-on-ESPN-57-Prelims-193918
Snippet: Women's Strawweights. Rayanne dos Santos (14-7, 0-1 UFC) vs. Puja Tomar (8-4, 0-0 UFC) ODDS: Dos Santos (-245), Tomar (+200)
---
Title: Rayanne Amanda Dos Santos vs Puja Tomar Prediction ... - YouTube
Link: https://www.youtube.com/watch?v=XtDmwvd5HwQ


 


Search results: Title: Rayanne dos Santos (Women's Strawweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/4601333/rayanne-dos-santos
Snippet: Fight History ; Dec 9, 2023, T. Alencar. L ; May 3, 2023, J. DeCoursey Championship Belt. W ; Jan 18, 2023, K. Saull. W ; Oct 13, 2022, V. Karolina. W.
---
Title: Rayanne dos Santos Record: 14-7-0 - Stats | UFC
Link: http://www.ufcstats.com/fighter-details/02bb48869eb7ac8f
Snippet: Rayanne dos Santos Record: 14-7-0 ... Career statistics: SLpM: 3.73; Str. Acc.: 36%; SApM: 4.63; Str. Def: 65%.
---
Title: Rayanne Amanda | MMA Fighter Page - Tapology
Link: https://www.tapology.com/fightcenter/fighters/188018-rayanne-amanda
Snippet: Given Name: Rayanne Amanda dos Santos; Pro MMA Record: 14-7-0 (Win-Loss-Draw); Nickname: N/A; Current Streak: 1 Loss; Age: 28 | Date of Birth: 1995.06.08 ...
---
Title: Rayanne dos Santos MMA Stats, Pictures, News, Videos, Biography
Link: https://www.sherdog.com/fighter/Rayanne-dos-Santos-230789
Sn

 

After conducting a search on the internet for public sentiment and expert predictions for the Rayanne dos Santos vs Puja Tomar fight, I found several sources providing insights. 

Expert predictions seem to favor Rayanne dos Santos, with odds in her favor. The general sentiment towards both fighters is not clearly outlined in the search results, but it appears that dos Santos is the favorite according to the experts and betting lines. The Reddit community does not seem to have a strong opinion on this matchup.

Overall, the expert predictions and odds suggest that Rayanne dos Santos is the favored fighter in this matchup.

 


Search results: Title: Rayanne dos Santos (Women's Strawweight) MMA Profile - ESPN
Link: https://www.espn.com/mma/fighter/_/id/4601333/rayanne-dos-santos
Snippet: Stats ; W-L-D. 14-7-0 ; (T)KO. 2-1 ; SUB. 8-1.
---
Title: Rayanne dos Santos MMA Career Stats - ESPN
Link: https://www.espn.com/mma/fighter/stats/_/id/4601333/rayanne-dos-santos
Snippet: Complete car

 

Based on the search results, Rayanne dos Santos has a professional MMA record of 14 wins and 7 losses. She has 2 wins by knockout, 8 wins by submission, and 4 wins by decision. Her last fight was against Puja Tomar on June 8, 2024, at UFC Fight Night. Puja Tomar, known as "The Cyclone," has a professional record of 8 wins and 4 losses and is currently on a four-fight winning streak.

 


Search results: Title: Rayanne dos Santos vs Talita Alencar Odds: Spreads, Lines & Totals
Link: https://www.bookmakersreview.com/ufc/matchups/rayanne-dos-santos-vs-talita-alencar/odds/
Snippet: The best odds today for Rayanne dos Santos is -150 at Everygame and for Talita Alencar , +135 at Bovada. The Over/Under line total is o2½ -165 at BetOnline.
---
Title: Rayanne dos Santos - UFC.com
Link: https://www.ufc.com/athlete/rayanne-dos-santos?language_content_entity=en
Snippet: Rayanne dos Santos is a Brazilian professional mixed martial artist in the UFC strawweight division. Get the latest UFC breaki